In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import pandas as pd

file_path = '/content/drive/My Drive/TCC - UFMS/TCC - UFMS - final/Corpus/merged_df.csv'
df = pd.read_csv(file_path)

In [4]:
df.head()

,Nome da Música,Artista,Gênero Musical,Letra da Música
0,Carolina,Seu Jorge,MPB,Carolina é uma menina bem difícil de esquecer ...
1,Epitáfio,Titãs,Rock,Devia ter amado mais Ter chorado mais Ter vist...
2,Lugar Ao Sol,Charlie Brown Jr.,Rock,"Que bom viver, como é bom sonhar E o que ficou..."
3,Relicário - Ao Vivo,Cássia Eller,Rock,É uma índia com colar A tarde linda que não qu...
4,Você Me Vira A Cabeça (Me Tira Do Sério),Alcione,Samba,"Você me vira a cabeça, me tira do sério Destró..."


In [5]:
import spacy

!python -m spacy download pt_core_news_lg
pln = spacy.load('pt_core_news_lg')
pln

from spacy.matcher import DependencyMatcher

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
from spacy import displacy
#testando
frases_primeira_pessoa = [
    "Eu sou bonita",
    "Nós somos muito bonitas",
    "Eu pareço fofa",
    "Eu estou ficando velha",
    "Eu fiquei brava"
]

df_teste = pd.DataFrame(frases_primeira_pessoa, columns=['Frase'])

print(df_teste)

for frase in df_teste['Frase']:
    doc = pln(frase)
    displacy.render(doc, style="dep", jupyter=True, options={"distance": 100})

                     Frase
0            Eu sou bonita
1  Nós somos muito bonitas
2           Eu pareço fofa
3   Eu estou ficando velha
4          Eu fiquei brava


In [ ]:
substantivo = [{"RIGHT_ID": "substantivo", "RIGHT_ATTRS": {"POS": "NOUN", "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}},
               {"LEFT_ID": "substantivo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj",  "LOWER": {"IN": ["eu", "nós"]}}},
               {"LEFT_ID": "substantivo", "REL_OP": ">", "RIGHT_ID": "auxiliar", "RIGHT_ATTRS": {"POS":{"IN":["AUX", "COP"]}}}]
#dep_matcher.add('substantivo', patterns=[substantivo])

In [7]:
dep_matcher = DependencyMatcher(vocab=pln.vocab)

vozpassiva = [{"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"MORPH": {"IN": ["Gender=Fem|Number=Sing|VerbForm=Part|Voice=Pass", "Gender=Fem|Number=Plur|VerbForm=Part|Voice=Pass"]}}},
              {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj:pass", "LOWER": {"IN": ["eu", "nós"]}}},
              {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "auxliar", "RIGHT_ATTRS": {"DEP": "aux:pass", "POS":"AUX"}}]

sujeitoauxadjetivo = [{"RIGHT_ID": "adjetivo", "RIGHT_ATTRS": {"POS": "ADJ", "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}},
                      {"LEFT_ID": "adjetivo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj",  "LOWER": {"IN": ["eu", "nós"]}}},
                      {"LEFT_ID": "adjetivo", "REL_OP": ">", "RIGHT_ID": "auxiliar", "RIGHT_ATTRS": {"POS":{"IN": ["AUX", "COP"]}}}]

ficar_parecer_tornar = [{"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"LEMMA":
                          {"IN": ["parecer", "ficar", "tornar"]}}},
                        {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj",  "LOWER": {"IN": ["eu", "nós"]}}},
                        {"LEFT_ID": "verbo", "REL_OP":">", "RIGHT_ID": "adjetivo", "RIGHT_ATTRS": {"DEP":"xcomp", "POS": "ADJ", "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}}]

sujeitoadjetivo = [{"RIGHT_ID": "noun", "RIGHT_ATTRS": { "LOWER": {"IN": ["eu", "nós"]}}},
                   {"LEFT_ID": "noun", "REL_OP":">", "RIGHT_ID": "adjetivo", "RIGHT_ATTRS": {"DEP":"amod", "POS": "ADJ", "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}}]


dep_matcher.add('vozpassiva', patterns=[vozpassiva])
dep_matcher.add('ficar_parecer_tornar', patterns=[ficar_parecer_tornar])
dep_matcher.add('sujeitoauxadjetivo', patterns=[sujeitoauxadjetivo])
dep_matcher.add('sujeitoadjetivo', patterns=[sujeitoadjetivo])

In [8]:
resultados = []

for index, row in df.iterrows():
    text = str(row['Letra da Música']).replace('\n', '. ')  # Substitui quebras de linha por ponto final
    doc = pln(text)
    dep_matches = dep_matcher(doc)
    for match in dep_matches:
        pattern_name = match[0]
        matches = match[1]
        if pln.vocab[pattern_name].text in ['vozpassiva', 'sujeitoauxadjetivo'] :
            inicio, final = matches[1], matches[0]
            frase = doc[inicio:final+1].text
            print('Padrão:', pln.vocab[pattern_name].text, 'Frase:', frase)
            resultados.append({'Padrão': pln.vocab[pattern_name].text,
                           'Frase': frase,
                           'Nome da Música': row['Nome da Música'],
                           'Nome do Artista': row['Artista'],
                           'Gênero Musical': row['Gênero Musical'],
                           'Letra da Música': text})
        elif pln.vocab[pattern_name].text in ['ficar_parecer_tornar']:
            inicio, final = matches[1], matches[2]
            frase = doc[inicio:final+1].text
            print('Padrão:', pln.vocab[pattern_name].text, 'Frase:', frase)
            resultados.append({'Padrão': pln.vocab[pattern_name].text,
                           'Frase': frase,
                           'Nome da Música': row['Nome da Música'],
                           'Nome do Artista': row['Artista'],
                           'Gênero Musical': row['Gênero Musical'],
                           'Letra da Música': text})
        elif pln.vocab[pattern_name].text == 'sujeitoadjetivo':
            inicio, final = matches[0], matches[1]
            frase = doc[inicio:final+1].text
            print('Padrão:', pln.vocab[pattern_name].text, 'Frase:', frase)
            resultados.append({'Padrão': pln.vocab[pattern_name].text,
                           'Frase': frase,
                           'Nome da Música': row['Nome da Música'],
                           'Nome do Artista': row['Artista'],
                           'Gênero Musical': row['Gênero Musical'],
                           'Letra da Música': text})

resultados_df = pd.DataFrame(resultados)
resultados_df.head()

Padrão: sujeitoauxadjetivo Frase: eu estava apaixonada
Padrão: sujeitoauxadjetivo Frase: eu estava apaixonada
Padrão: sujeitoauxadjetivo Frase: eu estava apaixonada
Padrão: sujeitoadjetivo Frase: eu (louca
Padrão: sujeitoadjetivo Frase: eu (louca
Padrão: sujeitoauxadjetivo Frase: eu (foi mais forte
Padrão: sujeitoauxadjetivo Frase: eu (foi mais forte
Padrão: sujeitoauxadjetivo Frase: eu (foi mais forte
Padrão: sujeitoauxadjetivo Frase: eu (foi mais forte
Padrão: sujeitoauxadjetivo Frase: Eu juro que eu quero me mudar daqui Vou atrás do progresso pra me adiantar Só quero ver minha família mais feliz
Padrão: sujeitoauxadjetivo Frase: eu tô preparada
Padrão: sujeitoauxadjetivo Frase: eu fosse mais nova
Padrão: ficar_parecer_tornar Frase: eu fico louca
Padrão: sujeitoauxadjetivo Frase: eu não tô sendo escroto
Padrão: sujeitoauxadjetivo Frase: eu não tô sendo escroto
Padrão: sujeitoauxadjetivo Frase: eu tô alegre
Padrão: sujeitoauxadjetivo Frase: eu era muito nova
Padrão: sujeitoauxadjetivo

,Padrão,Frase,Nome da Música,Nome do Artista,Gênero Musical,Letra da Música
0,sujeitoauxadjetivo,eu estava apaixonada,Meiga e abusada,Anitta,Pop,"Eu, posso conquistar tudo que eu quero Mas, fo..."
1,sujeitoauxadjetivo,eu estava apaixonada,Meiga e abusada,Anitta,Pop,"Eu, posso conquistar tudo que eu quero Mas, fo..."
2,sujeitoauxadjetivo,eu estava apaixonada,Meiga e abusada,Anitta,Pop,"Eu, posso conquistar tudo que eu quero Mas, fo..."
3,sujeitoadjetivo,eu (louca,"Pretin (Verão, Pt. 1)",Flora Matos,Rap,Pretin Desse jeito cê me deixa louca Tomando c...
4,sujeitoadjetivo,eu (louca,"Pretin (Verão, Pt. 1)",Flora Matos,Rap,Pretin Desse jeito cê me deixa louca Tomando c...


In [9]:
from google.colab import files
resultados_df.to_csv('resultados_df.csv', index=False)
files.download('resultados_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
resultados_df['Frase'] = resultados_df['Frase'].astype(str)
resultados_df['Frase'] = resultados_df['Frase'].str.strip()
resultados_df['Frase'] = resultados_df['Frase'].str.lower()
resultados_df['Frase'] = resultados_df['Frase'].str.replace(r'\s+', ' ', regex=True)
resultados_df = resultados_df.drop_duplicates(subset=['Frase', 'Nome da Música', 'Nome do Artista', 'Gênero Musical', 'Letra da Música'])
resultados_df.head()

,Padrão,Frase,Nome da Música,Nome do Artista,Gênero Musical,Letra da Música
0,sujeitoauxadjetivo,eu estava apaixonada,Meiga e abusada,Anitta,Pop,"Eu, posso conquistar tudo que eu quero Mas, fo..."
3,sujeitoadjetivo,eu (louca,"Pretin (Verão, Pt. 1)",Flora Matos,Rap,Pretin Desse jeito cê me deixa louca Tomando c...
5,sujeitoauxadjetivo,eu (foi mais forte,Tudo Aconteceu,MC Du Black,Funk,"Ei, Delacruz na voz Du Black na voz, na voz, n..."
9,sujeitoauxadjetivo,eu juro que eu quero me mudar daqui vou atrás ...,Deus é por nós,MC Marks,Funk,"(Ai, ai, ai, ai, ai) (Ai, ai, ai, ai, ai) (É o..."
10,sujeitoauxadjetivo,eu tô preparada,Desce Pro Play (PA PA PA),ZAAC,Funk,"[MC Zaac] Pa, pa, pa, pa, pa, pa, pa, pa, pa, ..."


In [11]:
resultados_df = resultados_df[resultados_df['Frase'].str.strip() != ""]
resultados_df.head()

,Padrão,Frase,Nome da Música,Nome do Artista,Gênero Musical,Letra da Música
0,sujeitoauxadjetivo,eu estava apaixonada,Meiga e abusada,Anitta,Pop,"Eu, posso conquistar tudo que eu quero Mas, fo..."
3,sujeitoadjetivo,eu (louca,"Pretin (Verão, Pt. 1)",Flora Matos,Rap,Pretin Desse jeito cê me deixa louca Tomando c...
5,sujeitoauxadjetivo,eu (foi mais forte,Tudo Aconteceu,MC Du Black,Funk,"Ei, Delacruz na voz Du Black na voz, na voz, n..."
9,sujeitoauxadjetivo,eu juro que eu quero me mudar daqui vou atrás ...,Deus é por nós,MC Marks,Funk,"(Ai, ai, ai, ai, ai) (Ai, ai, ai, ai, ai) (É o..."
10,sujeitoauxadjetivo,eu tô preparada,Desce Pro Play (PA PA PA),ZAAC,Funk,"[MC Zaac] Pa, pa, pa, pa, pa, pa, pa, pa, pa, ..."


In [12]:
resultados_df.shape

(1621, 6)

In [13]:
from collections import Counter

adjetivos = []
for frase in resultados_df['Frase']:
    doc = pln(frase)
    adjetivos += [token.text for token in doc if token.pos_ == "ADJ"]

contador_adjetivos = Counter(adjetivos)

ranking_adjetivos = pd.DataFrame(contador_adjetivos.items(), columns=['Adjetivo', 'Frequência'])
ranking_adjetivos = ranking_adjetivos.sort_values(by='Frequência', ascending=False).reset_index(drop=True)

print(ranking_adjetivos.head(10))

     Adjetivo  Frequência
0       louca         160
1       feliz          67
2  apaixonada          45
3     sozinha          41
4     cansada          37
5     carente          37
6       mesma          30
7      pronta          26
8         boa          25
9      alegre          23


In [15]:
print(ranking_adjetivos.head(20))

      Adjetivo  Frequência
0        louca         160
1        feliz          67
2   apaixonada          45
3      sozinha          41
4      cansada          37
5      carente          37
6        mesma          30
7       pronta          26
8          boa          25
9       alegre          23
10       linda          22
11      iguais          21
12          to          19
13       cheia          17
14       forte          17
15      errada          17
16       doida          17
17    inocente          16
18        boba          14
19     perdida          14


In [14]:
from google.colab import files
resultados_df.to_csv('resultados_df_sem_duplicata.csv', index=False)
files.download('resultados_df_sem_duplicata.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>